In [ ]:
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.8/722.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 57.9 MB/s eta 0:00:00


In [ ]:
import torch.nn as nn

In [ ]:
class Unet(nn.Module):
    def __init__(self):
        super(Unet, self).__init__()

        # Left side (contracting path)
        self.dwn_conv1 = dual_conv(1, 64)
        self.dwn_conv2 = dual_conv(64, 128)
        self.dwn_conv3 = dual_conv(128, 256)
        self.dwn_conv4 = dual_conv(256, 512)
        self.dwn_conv5 = dual_conv(512, 1024)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        #Right side  (expnsion path)
        #transpose convolution is used showna as green arrow in architecture image
        self.trans1 = nn.ConvTranspose2d(1024,512, kernel_size=2, stride= 2)
        self.up_conv1 = dual_conv(1024,512)
        self.trans2 = nn.ConvTranspose2d(512,256, kernel_size=2, stride= 2)
        self.up_conv2 = dual_conv(512,256)
        self.trans3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride= 2)
        self.up_conv3 = dual_conv(256,128)
        self.trans4 = nn.ConvTranspose2d(128,64, kernel_size=2, stride= 2)
        self.up_conv4 = dual_conv(128,64)

        #output layer
        self.out = nn.Conv2d(64, 2, kernel_size=1)

    def forward(self, image):

        #forward pass for Left side
        x1 = self.dwn_conv1(image)
        x2 = self.maxpool(x1)
        x3 = self.dwn_conv2(x2)
        x4 = self.maxpool(x3)
        x5 = self.dwn_conv3(x4)
        x6 = self.maxpool(x5)
        x7 = self.dwn_conv4(x6)
        x8 = self.maxpool(x7)
        x9 = self.dwn_conv5(x8)


        #forward pass for Right side
        x = self.trans1(x9)
        y = crop_tensor(x, x7)
        x = self.up_conv1(torch.cat([x,y], 1))

        x = self.trans2(x)
        y = crop_tensor(x, x5)
        x = self.up_conv2(torch.cat([x,y], 1))

        x = self.trans3(x)
        y = crop_tensor(x, x3)
        x = self.up_conv3(torch.cat([x,y], 1))

        x = self.trans4(x)
        y = crop_tensor(x, x1)
        x = self.up_conv4(torch.cat([x,y], 1))

        x = self.out(x)

        return x

In [ ]:
import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import Cityscapes
import torchvision.transforms as transforms

class CityscapesSegmentation(pl.LightningModule):
    def __init__(self, learning_rate=0.001):
        super(CityscapesSegmentation, self).__init__()

        # U-Net with 3 input channels (RGB) and the number of classes in Cityscapes
        self.model = Unet(3, 34)  # 34 classes in Cityscapes, can be adjusted
        self.criterion = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, targets = batch
        outputs = self(images)
        loss = self.criterion(outputs, targets)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return optimizer

    def train_dataloader(self):
        # Modify these paths and parameters as per your setup
        data_dir = "path_to_cityscapes_data"
        transform = transforms.Compose([
            transforms.Resize((256, 256)),  # Resize for faster training
            transforms.ToTensor(),
        ])

        dataset = Cityscapes(data_dir, split='train', mode='fine', target_type='semantic',
                             transform=transform, target_transform=transforms.ToTensor())
        dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
        return dataloader

# Training the Model
if __name__ == '__main__':
    model = CityscapesSegmentation()
    trainer = pl.Trainer(max_epochs=10)
    trainer.fit(model)


TypeError: ignored

In [ ]:
import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import Cityscapes
import torchvision.transforms as transforms

class CityscapesSegmentation(pl.LightningModule):
    def __init__(self, learning_rate=0.001):
        super(CityscapesSegmentation, self).__init__()

        # U-Net with 3 input channels (RGB) and the number of classes in Cityscapes
        self.model = UNet(3, 34)  # 34 classes in Cityscapes, can be adjusted
        self.criterion = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, targets = batch
        outputs = self(images)
        loss = self.criterion(outputs, targets)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return optimizer

    def train_dataloader(self):
        # Modify these paths and parameters as per your setup
        data_dir = "path_to_cityscapes_data"
        transform = transforms.Compose([
            transforms.Resize((256, 256)),  # Resize for faster training
            transforms.ToTensor(),
        ])

        dataset = Cityscapes(data_dir, split='train', mode='fine', target_type='semantic',
                             transform=transform, target_transform=transforms.ToTensor())
        dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
        return dataloader

# Training the Model
if __name__ == '__main__':
    model = CityscapesSegmentation()
    trainer = pl.Trainer(max_epochs=10)
    trainer.fit(model)
